# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [39]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sundays # Sunday schedule
import sys
import datetime as dt

# Customized functions to create the HTML for standard agenda items

In [40]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
def prayer(name, description, page):
    page.h2(description, align="center")
    page.p(name,align="center")    
    
def testimony(name, page):
    page.h2("Testimony", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")
        
# Calendar items are passed as a list of text strings
def calendar(textlist,page):
    page.hr()
    page.h1("Calendar Items", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

## Functions for the cleaning calendar

In [41]:
def nextsunday(n):
#""" Return next n Sundays, including today if it's Sunday"""
 today = dt.date.today()
 daynum = today.weekday() # Sunday is day #6

 if daynum == 6:
  basedate = today 
 else:
  basedate = today + dt.timedelta(6-daynum)
    
 sundays = []
 for i in range(n):
  nxtSunday = basedate + dt.timedelta(i*7)
  #sundays.append(nxtSunday.strftime("%B %d, %Y"))
  sundays.append(nxtSunday.strftime("%B %d"))
 return sundays

def nextsaturday(n):
#""" Return next n Saturdays, including today if it's Saturday"""
 today = dt.date.today()
 daynum = today.weekday() # Saturday is day #5

 if daynum == 5:
  basedate = today 
 else:
  basedate = today + dt.timedelta(5-daynum)
    
 saturdays = []
 for i in range(n):
  nxtSaturday = basedate + dt.timedelta(i*7)
  saturdays.append(nxtSaturday.strftime("%B %d"))
 return saturdays

In [42]:
roster = {
    "June 24" : "Sugar Land 1st",
    "July 01" : "Bradford, Arnold, Emerson, Markwalter",
    "July 08" : "Clark, Lambert, Blackburn, Ganci, Martin, Robinson",
    "July 15" : "Crandall, Blanco, Garcia, Moline, Thibault",
    "July 22" : "Crosby, Boyer, Griffin, Mike & Dawnette Moore, MacKinnon",
    "July 29" : "Maplewood 3rd Branch",
    "August 05" : "Draney, Buchanan, Hettinger, Morton, Tracey",
    "August 12" : "Leavitt, Chen, Hintze, Ozomah, Seegmiller",
    "August 19" : "Powell, Clements, Horowitz, Quam, Wightman",
    "August 26" : "Kaye Reynolds, Debow, Jeffery, Rasmussen, Winningham",
    "September 02" : "Roskelley, Ekstrom, Ocampo, Juarez, Jason & Deann Moore, Slater",
    "September 09" : "Arnold, Bradford, Emerson, Markwalter",
    "September 16" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "September 23" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "September 30" : "Maplewood 3rd Branch",
    "October 07" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "October 14" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "October 21" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "October 28" : "Clements, Horowitz, Powell, Quam, Wightman",
    "November 04" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "November 11" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
    "November 18" : "Arnold, Bradford, Emerson, Markwalter",
    "November 25" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "December 02" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "December 09" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "December 16" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "December 23" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "December 30" : "Maplewood 3rd Branch",
    "January 06" : "Clements, Horowitz, Powell, Quam, Wightman",
    "January 13" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "January 20" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
    "January 27" : "Yet to be assigned",
    "February 03" : "Yet to be assigned",
    "February 10" : "Yet to be assigned",
    "February 17" : "Yet to be assigned",
    "January 27" : "Bradford,  Arnold, Gutiérrez,  Markwalter,  Scott",
    "February 03" : "Clark,  Lambert,  Blackburn,  Martin,  Quam,  Robinson",
    "February 10" : "Crandall,  Blanco,  Ganci,  Garcia,  Moline,  Thibault",
    "February 17" : "Crosby,  Boyer,  Griffin, Mike & Dawnette Moore,  Thomas",
    "February 24" : "Draney,  Buchanan,  Hettinger,  Morton,  Tracey",
    "March 02" : "Leavitt,  Chen,  Hintze,  Ozomah,  Seegmiller",
    "March 09" : "Powell,  Clements,  Horowitz,  Rellaford,  Wightman",
    "March 16" : "Kaye  Reynolds,  Debow,  Jeffery,  Rasmussen,  Winningham",
    "March 23" : "Roskelley,  Ekstrom,  Ocampo,  Evans, Kyle & Vanessaa Moore,  Slater",
    "March 30" : "Maplewood 3rd  Branch",
    "April 06" : "Bradford,  Arnold,  Guiterrez,  Markwalter,  Scott",
    "April 13" : "Clark,  Lambert,  Blackburn,  Martin,  Quam,  Robinson",
    "April 20" : "Crandall,  Blanco,  Ganci,  Garcia,  Moline,  Thibault",
    "April 27" : "Crosby,  Boyer,  Griffin, Mike & Dawnette Moore,  Thomas",
    "May 04" : "Draney,  Buchanan,  Hettinger,  Morton,  Tracey",
    "May 11" : "Leavitt,  Chen,  Hintze,  Ozomah,  Seegmiller",
    "May 18" : "Powell,  Clements,  Horowitz,  Rellaford,  Wightman",
    "May 25" : "Kaye  Reynolds,  Debow,  Jeffery,  Rasmussen,  Winningham",
    "June 01" : "Roskelley,  Ekstrom,  Ocampo,  Evans, Kyle & Vanessaa Moore,  Slater",
    "ZZZ" : "End of List"
}

# Determine the Sunday date and what type of meeting

In [43]:
keydate = nextsunday(1)[0]
[meeting, format_date] = sundays.sunday_type(keydate)

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

In [44]:
title_choices = {'Sacrament Meeting': "Sugar Land Second Ward", 'Stake Conference': "Houston Texas South Stake"}
Title = title_choices.get(meeting, 'Sugar Land Second Ward')

#header_choices = {'Normal': "Sacrament Meeting", 'Fast': "Fast and Testimony Meeting",
#                  'Stake': "Stake Conference", "General" : "General Conference", "Ward" : "Ward Conference"}
#header = header_choices.get(meeting, 'Sacrament Meeting')

footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

#pagetitle("Sugar Land Second Ward", "General Conference", "October 1st, 2023", page)
#pagetitle("Sugar Land Second Ward", "Fast and Testimony Meeting", "February 4th, 2024", page)
#pagetitle("Sugar Land Second Ward", "Stake Conference", "December 2-3, 2023", page)
pagetitle(Title, meeting, format_date, page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

Christmas-themed keys are "Nativity1", "Annunc", "Birth", "Simon"

In [45]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art[1]), align="center")

## Officers

In [46]:
if meeting in ['Sacrament Meeting','Fast and Testimony Meeting']:
    #officer("Presiding","President Travis Bird", page)
    #officer("Presiding","President Mike Kennington", page)
    #officer("Presiding","President Ernie Carrasquillo", page)
    officer("Presiding","Bishop Joey Powell", page)
    #officer("Presiding","Bishop Joey Powell", page)
    #officer("Conducting","Bishop Joey Powell", page)
    #officer("Presiding","Brother Brent Leavitt", page)
    officer("Conducting","Brother Jared Draney", page)
    #officer("Presiding","Brother Brent Leavitt", page)
    #officer("Conducting","Brother Brent Leavitt", page)


## Opening and sacrament hymns

In [47]:
    music(111,"Opening Hymn",page)
    #prayer("Jack Reynolds", "Invocation", page)
    music(178,"Sacrament Hymn", page) 

## Variable part of the program

### If it's fast and testimony meeting, uncomment this section and comment out the sacrament meeting items

In [48]:
#Uncomment this line for fast and testimony meeting - logic should now handle this
if meeting == "Fast and Testimony Meeting":
    testimonies(page)

### For a sacrament meeting or ward conference, comment out the testimonies and use these sections as needed

In [49]:
if meeting in ["Sacrament Meeting", "Ward Conference", "Easter Service"]:
    # Use lines like these to form the speaking part of the program

    # Special code for the Primary and Christmas Programs
    #txtlist = (
    #    'Words and Music Celebrating the Birth and Mission of Jesus Christ',
    #    'Program presented by the Primary, Young Men and Young Women, Elders Quorum, and Relief Society',
    #    )
    #linklist = None

    #custom("Christmas Program", txtlist,linklist, page)
    #specialmusic("Primary Children",page, title="Fathers")
    #specialmusic("Primary ",page, title= "Primary Program - I'm trying to be like Jesus")

    #specialmusic("Sugar Land 2nd Ward Men's Chorus",page, accompanist="Dallin Arnold", title= "I Believe in Christ")
    #specialmusic("Ashley Parkinson",page, accompanist=None, title= "In Humility Our Savior")
    #testimony("President Ryman Tiu", page)
    #specialmusic("Enoch Poon",page, accompanist="Sarah Poon", title= "&Eacutel&eacutegie")
    #speaker("Ting Chen", page)
    #specialmusic("Joy Lawson",page, accompanist=None, title= "He Lives")
    
    #speaker("Emily Blanco",page)
    music(5,"Intermediate Hymn", page)
    #speaker("Dale Crosby",page)
    
    #prayer("","Benediction",page)
    



### Stake Conference Code - usually commented out

In [50]:
if meeting == "Stake Conference":
    # Special Code for Stake Conference
    txtlist = (
         "Bishopric, Ward Council, Counselors, Secretaries, and those who attend missionary and temple and family history correlation meetings",
         "In-person at Lexington"
    )
    linklist = None

    custom("Leadership Session, Saturday 2:00-4:30 pm", txtlist, linklist, page)

    txtlist = (
         "All Adults",
         "In-person at Lexington"
    )
    linklist = None

    custom("Adult Session, Saturday 7:00-8:30 pm", txtlist, linklist, page)

    txtlist = (
        "In-person at Lexington",
        "Via Zoom at Sienna and Rosenberg"
    )
    linklist = None

    custom("General Session, Sunday 10:00 am - 12:00 pm", txtlist, linklist, page)

### Closing hymn

In [51]:
if meeting in ["Sacrament Meeting", "Fast and Testimony Meeting", "Ward Conference", "Easter Service"]:
    music(85,"Closing Hymn",page)

## End matter - thought, announcements, calendar, links

In [52]:
# Spiritual thought
quote = ("The Lord isn't asking us to load up a handcart; He's asking us to fortify our faith.")
author = 'Elder M. Russell Ballard'
thought(quote, author, page)

# Make a list of announcements, then post to the page
txtlist = (
        "The sister missionaries have a new phone number: 346-901-8717",
        "The Houston Texas South Mission seeks senior couples to work in the office",
        "Volley Ball Wednesdays - Come join us on Wednesdays at 9PM and have fun!!!",
        "Send announcements to bvl2clerk@gmail.com")
announcements(txtlist, page)

callist = (
    "May 5th - HSS World Wide Devotional Broadcast for Young Adults at 7PM",
    "May 10th thru May 17th - Deseret Industries Pod at Lexington and Sienna buildings",
    "May 18th - HSS Stake Conference Adult Session 7:00 PM Lexington Building",
    "May 19th - HSS Stake Conference General Session 10:00 AM Lexington and Sienna buildings",
    "May 19th - Houston South Stake Seminary Graduation 6:30PM Lexington Building",
    "July 10th - Astros Friends & Family Night 7PM at Minute Maid Park See link below",
    "Share this event with family, friends, neighbors, co-workers and of course your ministering families!",
    "This will be a great opportunity to INVITE others to join us for an evening of fun at the ballpark",
    "Jully 10th-13th - Houstont Stake Young Women Camp at Caney Creek",
         )
calendar(callist, page)


# Cleaning Assignments

In [53]:
sats = nextsaturday(4)
cleaninglist = []
for sat in sats:
    cleaninglist.append(sat + ": " + roster[sat])
custom("Cleaning Assignments",cleaninglist, None, page)

### Preparedness Corner - under development

In [54]:
txtlist = (
"Emergency preparedness is the process of planning and preparing for potential hazards and disasters. Even though hurricane season for the Houston area is pretty much over, emergencies don't pay attention to the calendar.",
"This spring is as good a time as any to increase your family's preparedness. If you don't have 72-hour kits packed for everyone in your household (even if it's just you), let this guide you in your Christmas shopping!",
"If you're all prepared in that sense, now is a good time to wrap your outside faucets and do all else that you need to in order to be prepared for any upcoming freezes.",
"There are countless lists and resources online to give guidance and practical information on emergency preparedness. Go ahead and get started!",
    )
#linklist = {
#    ("Learn more online","https://www.churchofjesuschrist.org/study/manual/gospel-topics/food-storage?lang=eng")
#}
linklist = None
custom("Houston South Stake Emergency Preparedness Corner", txtlist,linklist, page)

In [55]:
# Make a list of links, then post
linklist = (
        ("Astros Friends and Family Night Wednesday, July 10th @7pm.","https://mlb.tickets.com/schedule/?agency=MLB_MPV&orgid=18#/sales_groups;salesGroupId=4940"),
        ("We need 30+ individuals for Jones Dickson Cemetery: The African American Heritage Project to help pick up trash, mow the grass, weed-eat, saw small branches, clear brush, and more. Please bring gloves and appropriate yard implements if you have them.", "https://www.justserve.org/projects/fb13bfc5-ea5b-4c45-89dc-47acddb2e80f"), 
        ("Let the missionaries know when you are available to help teach","https://docs.google.com/forms/d/1cVhEINFFESai0MLbgBA8euTyiMQsM8R1Sur_EQsyUrw/edit?usp=drive_web"),
        ("Get started with Temple and Family History work","https://sugar-land-2nd-ward-programs.github.io/TempleandFamilyHistoryGettingStarted.html"),
        ("See to Succeed a Houston South Stake service project","https://www.houstonhealth.org/services/clinical/see-to-succeed"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4#"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [56]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value
"""
End of script
"""

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html lang="en">
<head>
</head>
<body>
<br />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<h1 align="center">Sugar Land Second Ward</h1>
<h2 align="center">Fast and Testimony Meeting</h2>
<h2 align="center">May 5th</h2>
<p align="center"><img style="max-width:50%;height:auto;" src="https://assets.ldscdn.org/b2/7f/b27ffa65772bb46215dbdbc7163f6deb4011c8c2/pool_of_bethesda_carl_bloch.jpeg" /></p>
<p align="center"><b>Presiding:</b> Bishop Joey Powell</p>
<p align="center"><b>Conducting:</b> Brother Jared Draney</p>
<h2 align="center">Opening Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/rock-of-ages?lang=eng&amp;#39">#111, Rock of Ages</a></p>
<h2 align="center">Sacrament Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/o-lord-of-hosts?lang=eng&amp;#39">#178, O Lord of Hosts</a></p>
<h2 align="center">Bearin

'\nEnd of script\n'

In [57]:
meeting

'Fast and Testimony Meeting'